<a href="https://colab.research.google.com/github/pubalib22/learnGitBranching/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
real = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")


In [ ]:
real['label'] = 1
fake['label'] = 0

df = pd.concat([real, fake])
df = df[['title', 'text', 'label']]
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle

# Text cleaning
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['text_clean'] = df['text'].apply(clean_text)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['text_clean']).toarray()
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9856347438752784
[[4597   79]
 [  50 4254]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4676
           1       0.98      0.99      0.99      4304

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [ ]:
def predict_news(news_text):
    cleaned = clean_text(news_text)
    vec = tfidf.transform([cleaned]).toarray()
    pred = model.predict(vec)[0]
    return "Real" if pred == 1 else "Fake"
    predict_news("Breaking: Government introduces new policy to increase job opportunities.")